In [1]:
from choice_model import ChoiceModel
import os

folder = 'sim_graph'
base_url = 'http://127.0.0.1:11434'
# remove_type = 'Media Content'
remove_type = None
if not os.path.exists(folder):
    os.makedirs(folder)

old_model = ChoiceModel(base_url=base_url)
old_model.roll_back(0)
init_graph = old_model.graph.copy()
nodes = [n for n in init_graph.nodes if init_graph.nodes[n]['type'] != remove_type]
test_graph = init_graph.subgraph(nodes).copy()

old_model.save_graph(file_name=f"{folder}/{folder}_0.pkl")
old_model.visualize(file_name=f"{folder}/{folder}_0.html")

Saving graph to sim_graph/sim_graph_0.pkl


In [2]:
from choice_model import ChoiceModel


def predict(period,remove_type=None, k1=5,k2=1,top_k=10):
    model = ChoiceModel(base_url=base_url)
    prev_period_file = f'{folder}/{folder}_{period-1}.pkl'
    new_model = ChoiceModel(graph_path=prev_period_file,base_url=base_url) 
    graph = model.get_period(period)
    new_nodes = [n for n in graph.nodes if n not in list(new_model.graph.nodes)]
    new_nodes = [n for n in new_nodes if graph.nodes[n]['type'] != remove_type]
    new_nodes = sorted(new_nodes,key=lambda x: graph.nodes[x]['type'])
    test_graph = new_model.graph.copy()
    print("begin simulation",len(test_graph.nodes),len(test_graph.edges))
    test_graph,failed_nodes = model.predict_links(test_graph, new_nodes, file_name=f"{folder}/{folder}_{period}.pkl",
                                     choice_type=None,k1=k1,k2=k2,top_k=top_k)
    
    if len(failed_nodes) > 0:
        print("retrying failed nodes")
        test_graph,_ = model.predict_links(test_graph, failed_nodes,file_name=f"{folder}/{folder}_{period}.pkl",k1=k1,k2=k2,top_k=top_k)

    issolated_nodes = model.get_isolated_nodes(graph=test_graph)
    if len(issolated_nodes) > 0:
        print("retrying issolated nodes")
        test_graph,_ = model.predict_links(test_graph, issolated_nodes,file_name=f"{folder}/{folder}_{period}.pkl",k1=k1,k2=k2,top_k=top_k)
        test_graph = model.remove_isolated_nodes(graph=test_graph)
        model.save_graph(file_name=f"{folder}/{folder}_{period}.pkl",graph=test_graph)

    new_model = ChoiceModel(graph_path=f"{folder}/{folder}_{period}.pkl")
    new_model.visualize(file_name=f"{folder}/{folder}_{period}.html")
    print("end simulation",len(test_graph.nodes),len(test_graph.edges))
    PPI, CVI, DCI = new_model.evaluate(test_graph)
    print(f"Period {period} PPI: {PPI} CVI: {CVI} DCI: {DCI}")


In [3]:
predict(1)

begin simulation 99 178


  8%|▊         | 6/74 [00:31<05:45,  5.08s/it]

Saving graph to sim_graph/sim_graph_1.pkl


 11%|█         | 8/74 [00:47<07:07,  6.48s/it]

Saving graph to sim_graph/sim_graph_1.pkl


...

Saving graph to sim_graph/sim_graph_1.pkl


 26%|██▌       | 19/74 [02:15<11:12, 12.22s/it]

'answer'


 31%|███       | 23/74 [02:59<06:37,  7.79s/it]


KeyboardInterrupt: 

In [ ]:
for i in range(1,10):
    predict(i,remove_type=remove_type)